<a href="https://colab.research.google.com/github/Azimoj/WCS/blob/main/Dojo_W5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ookbook Recipes



You are given the table with titles of recipes from a cookbook and their page numbers. You are asked to represent how the recipes will be distributed in the book.
Produce a table consisting of three columns: left_page_number, left_title and right_title. The k-th row (counting from 0), should contain the number and the title of the page with the number
2×k in the first and second columns respectively, and the title of the page with the number
2×k+1 in the third column.
Each page contains at most 1 recipe. If the page does not contain a recipe, the appropriate cell should remain empty (NULL value). Page 0 (the internal side of the front cover) is guaranteed to be empty.





In [ ]:
# Jean-Simon

import pandas as pd
import numpy as np

facebook_friends = pd.read_csv('/work/facebook/facebook_friends.csv')

concatvalues =len(np.unique(np.concatenate([facebook_friends.user1.values,facebook_friends.user2.values])))
revert = facebook_friends.rename(columns= {'user1':'user2','user2':'user1'})
final = pd.concat([facebook_friends, revert],sort = False).drop_duplicates()

result = final.groupby('user1').size().to_frame('count').reset_index()
result['popularity_percent'] = 100*(result['count'] /concatvalues)

In [ ]:
df["right page"] = df.apply(lambda x : x[1] if x[0] %2 != 0 else "", axis=1)
df["left page"] = df.apply(lambda x : x[1] if x[0] %2 == 0 else "", axis=1)
df["left page number"] = df.apply(lambda x : x[0] if x[0] %2 == 0 else x[0]-1, axis=1)
df = df[["right page","left page","left page number"]]


df.groupby(by="left page number").agg(lambda x: ''.join(map(str, set(x))))

In [ ]:


df.sort_values(by='page_number', inplace=True)

max_page = df['page_number'].max()
page_range = range(0, max_page + 1, 2)

df_result = pd.DataFrame(index=page_range, columns=['left_page_number', 'left_title', 'right_title'])

for k in page_range:
    left_page_number = k
    left_title = df[df['page_number'] == k]['title'].values[0] if k in df['page_number'].values else ''

    right_page_number = k + 1
    right_title = df[df['page_number'] == right_page_number]['title'].values[0] if right_page_number in df['page_number'].values else ''

    df_result.loc[k] = [left_page_number, left_title, right_title]


df_result

Popularity Percentage



Find the popularity percentage for each user on Meta/Facebook. The popularity percentage is defined as the total number of friends the user has divided by the total number of users on the platform, then converted into a percentage by multiplying by 100.
Output each user along with their popularity percentage. Order records in ascending order by user id.
The 'user1' and 'user2' column are pairs of friends.





In [ ]:
# Jean-Simon

import pandas as pd
import numpy as np

facebook_friends = pd.read_csv('/work/facebook/facebook_friends.csv')

concatvalues =len(np.unique(np.concatenate([facebook_friends.user1.values,facebook_friends.user2.values])))
revert = facebook_friends.rename(columns= {'user1':'user2','user2':'user1'})
final = pd.concat([facebook_friends, revert],sort = False).drop_duplicates()

result = final.groupby('user1').size().to_frame('count').reset_index()
result['popularity_percent'] = 100*(result['count'] /concatvalues)
result



In [ ]:
# Start writing code here...
import pandas as pd
from collections import defaultdict

df_facebook = pd.read_csv('/work/facebook/facebook_friends.csv')
df_facebook

# Concaténer les colonnes user1 et user2 pour obtenir la liste complète d'utilisateurs
all_users = pd.concat([df_facebook['user1'], df_facebook['user2']])

# Calculer le nombre total d'utilisateurs sur la plateforme
total_users = len(all_users.unique())

total_users

# Créer un dataframe pour stocker le nombre d'amis pour chaque utilisateur
friend_count = pd.Series(all_users.value_counts(), name='friend_count').reset_index()
friend_count

# Renommer les colonnes pour la fusion
friend_count.columns = ['user_id', 'friend_count']

# Fusionner les dataframes pour obtenir le nombre d'amis pour chaque utilisateur
result_df = pd.merge(friend_count, df_facebook, left_on='user_id', right_on='user2', how='right')
result_df


# Calculer le pourcentage de popularité pour chaque utilisateur
result_df['popularity_percentage'] = (result_df['friend_count'] / total_users) * 100

# Arrondir les pourcentages
result_df['popularity_percentage'] = result_df['popularity_percentage'].round(2)

# Afficher le résultat trié par user_id
result_df = result_df[['user_id', 'popularity_percentage']].sort_values(by='user_id').reset_index(drop=True)
result_df


# Groupby par user_id
grouped_result = result_df.groupby('user_id').agg({'popularity_percentage': 'max'}).reset_index()

# Afficher le résultat
grouped_result

In [ ]:
# df con usuarios únicos
all_users = pd.unique(df[['user1', 'user2']].values.ravel('K'))
all_users
#número de amigos para cada usuario
friends_count = df['user1'].append(df['user2']).value_counts()
friends_count

# total de usuarios
total_users = len(all_users)
total_users

# porcentaje
popularity = (friends_count / total_users) * 100

popularity = popularity.sort_index()


print(popularity)

Risky Projects



Identify projects that are at risk for going overbudget. A project is considered to be overbudget if the cost of all employees assigned to the project is greater than the budget of the project.



You'll need to prorate the cost of the employees to the duration of the project. For example, if the budget for a project that takes half a year to complete is $10K, then the total half-year salary of all employees assigned to the project should not exceed $10K. Salary is defined on a yearly basis, so be careful how to calculate salaries for the projects that last less or more than one year.



Output a list of projects that are overbudget with their project name, project budget, and prorated total employee expense (rounded to the next dollar amount).





In [ ]:
# Jean-Simon

import pandas as pd
import numpy as np
from datetime import datetime

linkedin_projects = pd.read_csv('/work/linkedin/linkedin_projects.csv')
linkedin_emp_projects = pd.read_csv('/work/linkedin/linkedin_emp_projects.csv')
linkedin_employees = pd.read_csv('/work/linkedin/linkedin_employees.csv')



df = pd.merge(linkedin_projects, linkedin_emp_projects, how = 'inner',left_on = ['id'], right_on=['project_id'])
df1 = pd.merge(df, linkedin_employees, how = 'inner',left_on = ['emp_id'], right_on=['id'])

df1['project_duration'] = (pd.to_datetime(df1['end_date']) - pd.to_datetime(df1['start_date'])).dt.days

df_expense = df1.groupby('title')['salary'].sum().reset_index(name='expense')
df_budget_expense = pd.merge(df1, df_expense, how = 'left',left_on = ['title'], right_on=['title'])
df_budget_expense['prorated_expense'] = np.ceil(df_budget_expense['expense']*(df_budget_expense['project_duration'])/365)
df_budget_expense['budget_diff'] = df_budget_expense['prorated_expense'] - df_budget_expense['budget']
df_over_budget = df_budget_expense[df_budget_expense["budget_diff"] > 0]

result = df_over_budget[['title','budget','prorated_expense']]
result = result.drop_duplicates().sort_values('title')

In [ ]:
df3['start_date'] = pd.to_datetime(df3['start_date'])
df3['end_date'] = pd.to_datetime(df3['end_date'])

In [ ]:
df3['project_duration'] = (df3['end_date'] - df3['start_date']).dt.days
df3

In [ ]:
df = pd.merge(df1, df2, left_on='emp_id', right_on='id', how='left')
df

In [ ]:
df["salary"] = df["salary"]/365

In [ ]:
project_costs = df.groupby('project_id')['salary'].sum().reset_index()
project_costs

In [ ]:
df_all = pd.merge(df3, project_costs, left_on='id', right_on='project_id', how='inner')
df_all

In [ ]:
df_all['prorated_expense'] = round(df_all['salary'] * df_all['project_duration'])
df_all

In [ ]:
# Identificar proyectos sobre presupuesto

df_overbudget = df_all[['title', 'budget', 'prorated_expense']]
df_overbudget

In [ ]:
df_final_overbudget = df_overbudget[df_overbudget['prorated_expense'] > df_overbudget['budget']]
df_final_overbudget